# Validación de salinidad (ECe) con índices Sentinel-2

Este cuaderno carga puntos con índices (desde GEE) y mediciones de ECe, entrena modelos,
aplica **validación espacial por bloques**, calcula métricas (R², RMSE, MAE, RPD) y prepara
coeficientes/expresiones para aplicar el modelo en GEE.


In [0]:

# Si usas Colab, puedes necesitar instalar paquetes. Descomenta si es necesario.
# %pip install -q scikit-learn pandas matplotlib


In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Rutas de ejemplo (sube tus archivos o monta Drive)
indices_path = r"./dummy_ece_indices.csv"  # Reemplaza por 'indices_pts.csv' exportado desde GEE
field_path = r"./dummy_ece_indices.csv"    # Reemplaza por tu 'ece_terreno.csv' real cuando lo tengas

indices_df = pd.read_csv(indices_path)
field_df = pd.read_csv(field_path)

# En tu flujo real, 'indices_df' y 'field_df' serían distintos; aquí usamos el dummy para demo.
# Supón que comparten 'id' o ('lon','lat','date','depth_cm'). Haremos un merge por id para simplicidad.
df = pd.merge(indices_df, field_df[['id','ECe_dSm']], on='id', suffixes=('', '_y'))
df = df.rename(columns={'ECe_dSm': 'ECe'})
df = df[['id','lon','lat','depth_cm','date','NDVI','NDMI','SI','ECe']].dropna().copy()

print("Filas:", len(df))
df.head()


In [0]:

# Asignar bloques espaciales gruesos para validación sin fuga espacial
def make_blocks(lon, lat, size=0.001):  # ~100 m aprox; ajusta según AOI
    bx = np.floor((lon - lon.min()) / size).astype(int)
    by = np.floor((lat - lat.min()) / size).astype(int)
    return (bx * 10_000 + by).astype(int)

df['block_id'] = make_blocks(df['lon'].values, df['lat'].values, size=0.0015)

features = ['NDVI','NDMI','SI']
target = 'ECe'

X = df[features].values
y = df[target].values
groups = df['block_id'].values

# Modelos a probar
models = {
    'Linear': Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())]),
    'Ridge':  Pipeline([('scaler', StandardScaler()), ('model', Ridge(alpha=1.0))]),
    'RF':     Pipeline([('model', RandomForestRegressor(n_estimators=300, random_state=0, n_jobs=-1))])
}

def rpd(y_true, y_pred):
    sd = np.std(y_true, ddof=1)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return sd / rmse if rmse > 0 else np.nan

gkf = GroupKFold(n_splits=5)

results = []
preds_store = {{}}
for name, pipe in models.items():
    yhat = cross_val_predict(pipe, X, y, cv=gkf.split(X, y, groups=groups), n_jobs=-1)
    R2 = r2_score(y, yhat)
    RMSE = np.sqrt(mean_squared_error(y, yhat))
    MAE = mean_absolute_error(y, yhat)
    RPD = rpd(y, yhat)
    results.append((name, R2, RMSE, MAE, RPD))
    preds_store[name] = yhat

res_df = pd.DataFrame(results, columns=['Model','R2','RMSE','MAE','RPD']).sort_values('R2', ascending=False)
res_df


In [0]:

# Gráfico Observado vs Predicho para el mejor modelo
best = res_df.iloc[0]['Model']
yhat = preds_store[best]

plt.figure(figsize=(6,6))
plt.scatter(y, yhat, alpha=0.6)
mn, mx = min(y.min(), yhat.min()), max(y.max(), yhat.max())
plt.plot([mn, mx], [mn, mx])
plt.xlabel('ECe observado (dS/m)')
plt.ylabel('ECe predicho (dS/m)')
plt.title(f'Observed vs Predicted - {best}')
plt.tight_layout()
plt.show()


In [0]:

# Entrena el mejor modelo en todos los datos y exporta coeficientes si es lineal
best = res_df.iloc[0]['Model']
pipe = models[best]
pipe.fit(X, y)

if best in ('Linear','Ridge'):
    scaler = pipe.named_steps['scaler']
    model = pipe.named_steps['model']
    coefs = dict(zip(['NDVI','NDMI','SI'], model.coef_))
    intercept = model.intercept_
    print("Coeficientes (sobre variables estandarizadas):", coefs)
    print("Intercept:", intercept)

    # Para aplicar en GEE, calcula fórmula explícita en términos originales:
    mu = dict(zip(['NDVI','NDMI','SI'], scaler.mean_))
    sd = dict(zip(['NDVI','NDMI','SI'], scaler.scale_))

    # ECe = b0 + sum(bi * (Xi - mu_i)/sd_i)
    # Reescribe como ECe = A0 + a_NDVI*NDVI + a_NDMI*NDMI + a_SI*SI
    a = {k: (coefs[k]/sd[k]) for k in coefs}
    A0 = intercept - sum((coefs[k]*mu[k]/sd[k]) for k in coefs)
    print("Expresión no estandarizada:")
    print("ECe = {:.6f} + {:.6f}*NDVI + {:.6f}*NDMI + {:.6f}*SI".format(A0, a['NDVI'], a['NDMI'], a['SI']))

else:
    # Para RF, la aplicación por pixeles es más cómoda en Python raster o en GEE con TF/EE (no cubierto aquí).
    if hasattr(pipe.named_steps['model'], 'feature_importances_'):
        fi = dict(zip(['NDVI','NDMI','SI'], pipe.named_steps['model'].feature_importances_))
        print("Importancias (RF):", fi)


In [0]:

# Clasificación por umbrales de ECe (ajusta a tu cultivo/contexto)
# Ejemplo: <2 (baja), 2-4 (leve), 4-8 (moderada), 8-16 (alta), >16 (extrema)
bins = [-np.inf, 2, 4, 8, 16, np.inf]
labels = ['baja','leve','moderada','alta','extrema']
best = res_df.iloc[0]['Model']
yhat = preds_store[best]
classes = pd.cut(yhat, bins=bins, labels=labels)

pd.DataFrame({'ECe_obs': y, 'ECe_pred': yhat, 'class_pred': classes}).head()
